In [ ]:
using Images

In [ ]:
global atype = KnetArray{Float32}

In [ ]:
include("src/newmacnetwork.jl")

In [ ]:
feats,qstsns,dics = loadDemoData()

In [ ]:
_,wrun,r,_,p = loadmodel("models/demo.jld")

In [ ]:
qvoc,avoc,i2w,i2a = dics

In [ ]:
r          = randperm(length(qstns))[1]
inst       = qstns[r]
feat       = feats[:,:,:,r:r]
question   = Array{Int}(inst[2].+1)
answer     = inst[3].+1;
family     = inst[4];
results,prediction = visualize(wrun,r,feat,question);

In [ ]:
img = load("data/demo/CLEVR_v1.0/images/val/$(inst[1])")

In [ ]:
textq  = i2w[question];
println("Question:\n",join(textq," "))
texta  = i2a[answer];
println("Answer: $(texta)\nPrediction: $(i2a[prediction]) ")

In [ ]:
userinput = readline(STDIN)
words = split(userinput)
question = [ w2i[wr] for wr in words] .+ 1
results,prediction = forward(w,r,feat,question);
println("Prediction: $(i2a[prediction])")

In [ ]:
s_y,s_x = size(img)./14
for k=1:12
    α = results["w_attn_$(k)"][:]
    println("step_$(k) most attn. wrds: ",i2w[question[sortperm(α;rev=true)[1:2]]])
    flush(STDOUT)
    display([RGB{N0f8}(α[i],α[i],α[i]) for i=1:length(α)]);
    hsvimg = convert.(HSV,img);
    attn = results["KB_attn_$(k)"]
    for i=1:14,j=1:14
        rngy          = floor(Int,(i-1)*s_y+1):floor(Int,min(i*s_y,320))
        rngx          = floor(Int,(j-1)*s_x+1):floor(Int,min(j*s_x,480))
        hsvimg[rngy,rngx]  = scalepixel.(hsvimg[rngy,rngx],attn[sub2ind((14,14),i,j)])
    end
    display(hsvimg)
end